In [24]:
import dashscope
from http import HTTPStatus
dashscope.api_key="sk-632f5cf28f0a43719096801cd7c2e61a"

def call_with_messages(question,item1,item2,item3):
    messages = "客户的问题是：\n" + question + "\n根据问题所匹配的条例为1：\n" + item1 + "\n根据问题所匹配的条例为2：\n" + item2 + "\n根据问题所匹配的条例为3：\n" + item3
    messages = [{'role': 'system', 'content': '你是一个医疗保险条例解读专家'},
                {'role': 'system', 'content': '请为用户根据下面给出的保险条例来回答客户的问题'},
                {'role': 'user', 'content': messages}]

    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_turbo,
        messages=messages,
        result_format='message',  # 将返回结果格式设置为 message
    )
    content = response.output.choices[0]["message"]["content"]
    return content

def call_with_simple(question):
    content = "我在进行一个文字检索任务，请帮我从客户给出的问题中提取三个医疗保险咨询相关的关键词用于条文检索。\n 请注意，需要忽略如经典个人计划A产品和医疗保险等产品名。\n 请注意，只回答我关键词，不要其他内容。\n 【具体问题如下】：\n" + question
    messages = [{'role': 'system', 'content': '你是一个有用的助手'},
                {'role': 'user', 'content': content}]

    response = dashscope.Generation.call(
        model='qwen-max-0428',
        messages=messages
    )
    return response["output"]["text"]

import json
data_list = []
with open("./RAG_data/Contract_list_bge/Contract_list.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
        # 每一行都是一个JSON字符串，使用json.loads转换为字典
        data_list.append(json.loads(line.strip()))

In [18]:
res=call_with_simple("如果我接下来要前往欧洲国家外派工作半年的时间，是否可以购买你们的经典个人产品吗")
print(res)

外派工作、欧洲国家、半年时间


In [4]:
import json
data_list = []
with open("..//RAG_data/QA_jsonl/QA_list.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
        # 每一行都是一个JSON字符串，使用json.loads转换为字典
        data_list.append(json.loads(line.strip()))

from pyserini.search.lucene import LuceneSearcher
from pyserini.search.faiss import FaissSearcher, AutoQueryEncoder
from pyserini.search.hybrid import HybridSearcher

ssearcher = LuceneSearcher('../indexes/Insurance/bm25_index/QA_index')
encoder = AutoQueryEncoder('../Model/mixed_model_1')
dsearcher = FaissSearcher(index_dir = '../indexes/Insurance/dense_index/QA_index',query_encoder = encoder)
hsearcher = HybridSearcher(dsearcher, ssearcher)

In [7]:
question = "若在等待期间不幸罹患已排除在保险范围内的疾病，但在等待期结束后再次发病，这种疾病是否会被视为既往病症而不予保障？"
hits = dsearcher.search(query=question, k=1)
item = data_list[int(hits[0].docid)]['contents']
print(item)


如等待期内罹患某项疾病投保时除外承保了但过了等待期再次患上相同疾病，该疾病会算成既往症不保障吗?


In [3]:
data_list

[{'id': 0, 'contents': '2023版和即时生效版最大区别在哪?'},
 {'id': 1, 'contents': '是否所有的门急诊医疗保险金都可以赔付?'},
 {'id': 2, 'contents': '互联网问诊和购药费用如何使用?'},
 {'id': 3, 'contents': '健康检查及疫苗责任，就医的医院范围?'},
 {'id': 4, 'contents': '健康检查都包含什么,儿保服务次数和内容有哪些?'},
 {'id': 5, 'contents': '哪些项目不属于健康检查责任?'},
 {'id': 6, 'contents': '防控近视保障对镜片有什么要求吗?'},
 {'id': 7, 'contents': '去指定私立网络就诊，忘了出示直付保险卡，可不可以事后提供收据报销?'},
 {'id': 8, 'contents': '此产品保证续保吗?'},
 {'id': 9, 'contents': '此产有犹豫期退保嘛?'},
 {'id': 10,
  'contents': '“京东安联成长优享”事先授权条款有写被保险人在接受治疗前要向贵司提前5个工作日申请但如果情况很紧急，被保险人需要马上住院或者急诊需要住院怎么办，可以先住院治疗再申请吗?'},
 {'id': 11,
  'contents': '“京东安联成长优享”小海星计划中责任免除第四点第二条写到“本附加合同约定医疗机构外产生的医疗费用(持本附加合同约定医疗机构主治医生处方和院外购药原因说明的院外购药费用不在此限)”，第三方管理机构风石健康说这是不赔付的，请问这一条怎么解读呢。'},
 {'id': 12, 'contents': '京东安联成长优享2023”是否可以进行院外购药，责任免除没有查到相关描述，请问院外购药赔还是不赔?'},
 {'id': 13,
  'contents': '“京东安联成长优享2023”健康检查及疫苗责任中约定需要选定一个医疗品牌就医，请问如果是前往公立医院接种疫苗，也只能去医疗网络中的其中一家吗?'},
 {'id': 14, 'contents': '京东安联成长优享2023”理赔时对住院病房有限制吗，比如只能标准单人间或者高档病房?'},
 {'id': 15,
  'contents': '京

In [15]:
hits[1]

DenseSearchResult(docid='25', score=14.496565)

In [9]:
hits

[DenseSearchResult(docid='20', score=14.685943)]

In [ ]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.search.faiss import FaissSearcher, AutoQueryEncoder
from pyserini.search.hybrid import HybridSearcher

ssearcher = LuceneSearcher('./indexes/Insurance/Contract_list')
encoder = AutoQueryEncoder('../Model/bge-m3')
dsearcher = FaissSearcher(index_dir = "./indexes/bge/Contract",query_encoder = encoder)
hsearcher = HybridSearcher(dsearcher, ssearcher)

In [28]:
question_list = ["如果我接下来要前往欧洲国家外派工作半年的时间，是否可以购买你们的经典个人产品吗？","我购买了你们的精选个人产品，我想知道你们涵盖多久的住院康复治疗费用呢？",
                 "我购买了你们的经典个人计划A产品，护理费是可以全额理赔的是吗？存在什么限制吗？","我购买了你们的经典个人计划A产品，我最近需要拔智齿，请问这项治疗是否可以给我赔付？",
                 "我购买了你们的经典个人计划B产品，我最近想要做理疗，请问你们对于理疗次数或者其他条件有限制吗，我去哪里做理疗都可以给我赔付吗？","你们的经典个人产品是否涵盖质子重离子放射治疗疗法？还涵盖哪些肿瘤治疗方法？"
                ]
output_file = './model_output.txt'
with open(output_file, 'w', encoding='utf-8') as file:
    for question in question_list:
        question_word = call_with_simple(question)
        hits = hsearcher.search(question_word, k=3, alpha=0.3)
        item1 = data_list[int(hits[0].docid)]['contents']
        item2 = data_list[int(hits[1].docid)]['contents']
        item3 = data_list[int(hits[2].docid)]['contents']
        ans = call_with_messages(question, item1, item2, item3)
        
        # 同时打印到控制台和写入文件
        print("输入的问题是：", question)
        print("根据问题识别的关键词为：", question_word)
        print("AI回答：", ans)
        print("相关条款1为：", item1)
        print("相关条款2为：", item2)
        print("相关条款3为：", item3)
        print("本问题结束！Question END!_____________________________________________________________________\n")
        
        # 写入文件，注意这里用file.write并且不包含换行符'\n'，因为在字符串中已经包含了换行
        file.write("输入的问题是：{}\n".format(question))
        file.write("根据问题识别的关键词为：{}\n".format(question_word))
        file.write("AI回答：{}\n".format(ans))
        file.write("相关条款1为：{}\n".format(item1))
        file.write("相关条款2为：{}\n".format(item2))
        file.write("相关条款3为：{}\n".format(item3))
        file.write("本问题结束！Question END!_____________________________________________________________________\n\n")  # 每个问题间保留空行

print("所有结果已成功保存至文件：", output_file)

输入的问题是： 如果我接下来要前往欧洲国家外派工作半年的时间，是否可以购买你们的经典个人产品吗？
根据问题识别的关键词为： 外派工作、欧洲国家、半年。
AI回答： 您好，根据您提供的信息，条款2和3似乎与您的问题相关。条款2规定了可投保的人群范围，包括十八至七十周岁的能正常工作的自然人，以及他们的配偶和符合条件的子女。条款3提到在保险期间内，被保险人需要在中国大陆、香港、澳门及台湾地区累计居住达到保险期间的三分之二。

如果您计划前往欧洲国家外派工作半年，这可能不符合条款3中关于居住地的要求。通常情况下，经典个人产品的保障范围限于中国大陆及港澳台地区。因此，您可能无法直接购买这款产品，除非您能够确保在保险期间内大部分时间仍然在中国大陆或港澳台地区。建议您联系保险公司确认其具体政策，看是否可以做出相应的例外安排或者选择其他适合您的海外旅行保险产品。
相关条款1为： 第6条:除本合同另有约定外，本合同涉及的币别为人民币。
相关条款2为： 第2条:投保时年龄在十八周岁（含）至七十周岁（含）之间，能正常工作、劳动或生活的自然人，经保险人同意，可作为主被保险人。
	投保时年龄不超过七十周岁，能正常工作、劳动或生活的主被保险人配偶，经保险人同意，可作为附属被保险人。
	投保时年龄不超过二十一周岁（为全日制在校学生，或存在身体缺陷或智力障碍的，年龄不超过二十六周岁）身体健康，能正常学习且经济上完全依赖主被保险人的未婚子女，经保险人同意，也可作为附属被保险人。
	投保时年龄在七十一周岁（含）至九十九周岁（含）之间的自然人（子女除外），同时满足以下三个条件的，才可作为本保险的主被保险人或附属被保险人：
（一）非首次投保本保险；
	（二）上一个保险期间届满，投保人重新向保险人投保本保险；
	（三）本合同的保险期间与上一个保险合同的保险期间之间连续不间断。
主被保险人和附属被保险人统称或者泛称为“被保
　险人”，单称或者特称为“每一被保险人”、“该被保险人”或者“被保险人本人”。
	在保险期间内，被保险人应当在中国大陆、香港特别行政区、澳门特别行政区以及台湾地区（以下简称“大陆及港澳台”）居住累计达到保险期间的三分之二；未满足该条件的，保险人有权据此调整承保条件或者保险费率。
相关条款3为： 第26条:的约定。
本问题结束！Question END!___________________